# Building a chatbot

### what we will build will only use the language models to have a conversation

>enabale chatbot bot expirence over an external source of data using conversational RAG

>will build a chatbot that can do actions using agents

In [8]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")


In [13]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model = ChatOpenAI(model="gpt-3.5-turbo",openai_api_key=openai_api_key)

#model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10feaa260>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10feabfd0>, root_client=<openai.OpenAI object at 0x10ee80580>, root_async_client=<openai.AsyncOpenAI object at 0x10feaa290>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [14]:
from langchain_core.messages import HumanMessage
response = model.invoke([HumanMessage(content="Hi, I am Sanatan Khemariya, a B.Tech 3rd year student specializing in micro degrees in AI/ML and majoring in CS.")])

print(response)

content="Hello Sanatan! It's great to hear about your academic pursuits in AI/ML and Computer Science. How can I assist you today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 41, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-85362405-fcc8-48ff-a682-01def5e5aafd-0' usage_metadata={'input_tokens': 41, 'output_tokens': 29, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [15]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, I am Sanatan Khemariya, a B.Tech 3rd year student specializing in micro degrees in AI/ML and majoring in CS."),
        AIMessage(content="Hello Sanatan! It's great to hear about your academic pursuits in AI/ML and Computer Science. How can I assist you today?"),
        HumanMessage(content="I am looking for an internship Can you help me with that?"),
    ]
)


AIMessage(content="Of course! Here are a few tips to help you find an internship in your field:\n\n1. Update your resume and LinkedIn profile to highlight your skills and experience in AI/ML and Computer Science.\n\n2. Reach out to your college's career services office for guidance and resources on finding internships.\n\n3. Network with professionals in your field through LinkedIn, industry events, and networking events to learn about potential internship opportunities.\n\n4. Check job boards, company websites, and social media platforms for internship postings in AI/ML and Computer Science.\n\n5. Consider reaching out to companies directly to inquire about potential internship opportunities, even if they don't have any posted.\n\nGood luck with your internship search, Sanatan! If you need any more assistance, feel free to ask.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 155, 'prompt_tokens': 90, 'total_tokens': 245, 'completion_token

### Message History
we can use a message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore.
Future interactions will then load those messages and pass them into chain as part of the input.

In [17]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

#this function will get the stored chat history based on the sessions id to distinguist between different chat sessions

history = {}
def get_session_history(sessions_id:str)->BaseChatMessageHistory:
    if sessions_id not in history:
        history[sessions_id] = ChatMessageHistory() ## whatever the chat is happening it will automatically go inside the history dictionary to store the chat history
    return history[sessions_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [25]:
config = {"configurable":{"session_id":"chat_1"}}

In [27]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Hi, I am Sanatan Khemariya, a B.Tech 3rd year student specializing in micro degrees in AI/ML and majoring in CS."),
    ],
    config=config
)

In [28]:
response.content

"That's great! It sounds like you have a strong foundation in technology and are exploring some cutting-edge fields like AI/ML. If you have any questions or need assistance with your studies or projects in these areas, feel free to ask. I'm here to help!"

In [29]:
with_message_history.invoke(
    [
        HumanMessage(content="who i am and what i do"),
    ],
    config=config
)

AIMessage(content="You are Sanatan Khemariya, a B.Tech 3rd year student specializing in micro degrees in AI/ML and majoring in Computer Science. This means you are currently pursuing a degree in technology with a focus on artificial intelligence and machine learning, as well as studying various aspects of computer science. \n\nAs a student, you are likely engaged in learning about programming, algorithms, data structures, and other key topics in your field. You may also be working on projects or assignments related to AI/ML, applying theoretical knowledge to real-world problems. If you have any specific questions or need guidance on any academic or technical topics, feel free to share and I'll do my best to assist you.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 144, 'prompt_tokens': 183, 'total_tokens': 327, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_to

In [31]:
### change the config to chat2 to change the session id to check if the chat history is stored or not

config_1 = {"configurable":{"session_id":"chat_2"}}

In [33]:
response=with_message_history.invoke(
    [
        HumanMessage(content="do you know me?"),
    ],
    config=config_1
)
response.content

'As an AI language model, I do not have the ability to know or remember specific individuals. Each interaction I have with a user is based solely on the input provided to me during that conversation. I am here to assist you with any questions or tasks you may have to the best of my ability based on the information available to me.'

# Prompt Templates

Promopt templates help to turn raw user information into a format that the LLM can work with . in this case , the raw user input is just the message , which we are passing to the LLM . Let's now make that a bit more complicated .

First . let's add in a system message with some custom instructions(but still taking messages as input).Next, we'll add in more input besides just the messages.

In [72]:
from langchain_core.prompts import ChatMessagePromptTemplate,MessagesPlaceholder,ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a helpful assistant.answer all the question to the best of your ability"
         ),
        MessagesPlaceholder(variable_name="messages")

    ]
)
chain = prompt|model

In [73]:
chain.invoke({"messages":[HumanMessage(content="Hi, I am Sanatan Khemariya, a B.Tech 3rd year student specializing in micro degrees in AI/ML and majoring in CS.")]})

AIMessage(content="Hello, Sanatan! It's great to meet you. How can I assist you with your studies or any other questions you might have related to AI/ML, CS, or anything else?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 60, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2bc1903a-e602-4a7d-be91-eb4e4a983bc8-0', usage_metadata={'input_tokens': 60, 'output_tokens': 40, 'total_tokens': 100, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [74]:
## invokeing with chat message history
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [75]:
config_2 = {"configurable": {"session_id": "chat_3"}}

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="Hi, I am Sanatan Khemariya, a B.Tech 3rd year student specializing in micro degrees in AI/ML and majoring in CS.")],
        
    },
    config=config_2
    
)

print(response)


content="Hello Sanatan! It's great to hear about your academic background and specialization. How can I assist you today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 1816, 'total_tokens': 1840, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-00438d73-a0c0-4f89-8154-bb24c70187d1-0' usage_metadata={'input_tokens': 1816, 'output_tokens': 24, 'total_tokens': 1840, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [76]:
response.content

"Hello Sanatan! It's great to hear about your academic background and specialization. How can I assist you today?"

In [77]:
response = with_message_history.invoke(
    [
        HumanMessage(content="what is my name?"),
    ],
    config=config_2
)

In [78]:
response.content

'Your name is Sanatan Khemariya. How can I assist you further, Sanatan?'

### Adding language specific responses

In [163]:
## we can make it more compleax (as we can insert a language parameter to talk in certain message)
from langchain_core.prompts import ChatMessagePromptTemplate,MessagesPlaceholder,ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions in {language}. Always respond in {language}."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt|model

### Response in desired language

In [99]:
response = chain.invoke({"messages":[HumanMessage(content="Hi, I am Sanatan Khemariya, a B.Tech 3rd year student specializing in micro degrees in AI/ML and majoring in CS.")],"language":"Hindi"})
response.content

'नमस्कार, Sanatan Khemariya! आपका स्वागत है। आपकी वर्णित में मैं आपको सहायक बनाने में भविष्यवाणी कर सकता हूँ। कैसे मदद कर सकता हूँ?'

### Function to set default languge as english if no parameter is passed

In [161]:
# Dictionary to store both chat history and session language
session_data = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    """
    Retrieves chat history for a given session, initializing if necessary.
    """
    if session_id not in session_data:
        session_data[session_id] = {"history": ChatMessageHistory(), "language": "English"}  # Default language
    return session_data[session_id]["history"]

def set_session_language(session_id: str, language: str):
    """Ensure each session correctly remembers the preferred language."""
    if session_id not in session_data:
        session_data[session_id] = {"history": ChatMessageHistory(), "language": language}
    else:
        # ✅ Only update language if it's not already set
        if "language" not in session_data[session_id]:
            session_data[session_id]["language"] = language



In [162]:
def invoke_with_language(session_id: str, messages, language=None):
    """Handles chatbot invocation while ensuring memory & language persistence."""
    if session_id not in session_data:
        set_session_language(session_id, "English")  # ✅ Default to English

    if language is None:
        language = session_data[session_id]["language"]  # ✅ Retrieve stored language

    set_session_language(session_id, language)  # ✅ Ensure language is remembered

    response = with_message_history.invoke(
        {
            "messages": messages,
            "language": language  # ✅ Ensures correct language is passed
        },
        config={"configurable": {"session_id": session_id}}
    )

    return response



In [ ]:
config_4 = {"configurable":{"session_id":"chat_4"}}

In [125]:
response1 = invoke_with_language(
    session_id="chat_3",
    messages=[HumanMessage(content="Hi, I am Sanatan Khemariya, a B.Tech 3rd year student specializing in micro degrees in AI/ML and majoring in CS.")],
    language="Hindi"
)

print("Response in Hindi:", response1.content)

Response in Hindi: नमस्ते, सनातन खेमरिया। आपका स्वागत है। आपका किसी विषय पर सवाल है क्या?


In [126]:
response1.content



'नमस्ते, सनातन खेमरिया। आपका स्वागत है। आपका किसी विषय पर सवाल है क्या?'

In [127]:
response2 = invoke_with_language(
    session_id="chat_3",
    messages=[HumanMessage(content="What's my name?")]
)

print("Response in Hindi:", response2.content)  # ✅ Should still reply in Hindi

Response in Hindi: आपका नाम सनातन खेमरिया है।


In [129]:
response2 = invoke_with_language(
    session_id="chat_3",
    messages=[HumanMessage(content="What's my name?")],
    language="English"
)

print("Response in Hindi:", response2.content)  # changed the language to english now

Response in Hindi: Your name is Sanatan Khemariya.


In [130]:
response2 = invoke_with_language(
    session_id="chat_3",
    messages=[HumanMessage(content="what is my profession")]
    
)

print("Response in Hindi:", response2.content)  # now should reply in englis

Response in Hindi: You are a B.Tech 3rd year student specializing in micro degrees in AI/ML and majoring in CS.


In [133]:
response2 = invoke_with_language(
    session_id="chat_3",
    messages=[HumanMessage(content="what i asked previously from you")],
    language="Hindi"
    
)

print("Response in Hindi:", response2.content)  # now should reply in hindi with remembering previous context

Response in Hindi: You previously asked me, "What's my name?"


# Managing the conversation history

One more important concept to understand when builsding chatbots is how  to manage the conversation history.if left unmanaged , the list of messages will grow unbounded and potentialy overflow the context window of the the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.


the trim_messages helper to reduce how many messages we're sending to the model. the trimmer allows us to specify how many tokens we want to keep , along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [141]:
# Custom token counter function (alternative to model.get_num_tokens)
def token_counter(messages):
    """Simple token counter using text length approximation"""
    return sum(len(msg.content.split()) for msg in messages)

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages

trimmer = trim_messages(
    max_tokens=1000,  
    strategy="last",   
    include_system=True,  
    allow_partial=False,
    start_on="human",
    token_counter=token_counter #work on the token counter function
)

messages = [
    HumanMessage(content="Hi, I am Bob."),
    SystemMessage(content="Hi Bob! How can I assist you today?"),
    HumanMessage(content="I need help with my resume."),
    AIMessage(content="Of course! What field are you applying for?"),
    HumanMessage(content="Software Engineering."),
    AIMessage(content="Great! Do you have any experience in programming languages like Python or Java?"),
    HumanMessage(content="Yes, I have worked with both."),
    AIMessage(content="That's great! I suggest highlighting your experience with Python and Java in your skills section."),
    HumanMessage(content="Should I include personal projects?"),
    AIMessage(content="Absolutely! Personal projects show initiative and problem-solving skills."),
    HumanMessage(content="How should I format my resume?"),
    AIMessage(content="Use a clean, structured layout with sections like Summary, Skills, Experience, and Projects."),
    HumanMessage(content="Thanks! That helps a lot."),
    AIMessage(content="You're welcome! Let me know if you need further guidance."),
]



In [144]:
trimmer.invoke(messages)

[HumanMessage(content='Hi, I am Bob.', additional_kwargs={}, response_metadata={}),
 SystemMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I need help with my resume.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Of course! What field are you applying for?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Software Engineering.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Great! Do you have any experience in programming languages like Python or Java?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Yes, I have worked with both.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's great! I suggest highlighting your experience with Python and Java in your skills section.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Should I include personal projects?', additional_kwargs={}, response_meta

In [186]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)

response = chain.invoke({"messages": messages + [HumanMessage(content="can you give me a cool and catchy nick name")],
              "language": "Hindi"
              })
response.content

'बिलकुल! क्या मुझे आपकी कुछ पसंदीदा चीजों के बारे में बता सकते हैं?'

In [187]:
#chain with session history
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

# Define session configuration
config_5 = {"configurable": {"session_id": "chat_5"}}


In [194]:

response = with_message_history.invoke(
    {
        "messages": [
            HumanMessage(content="Hi, I am Bob."),
            SystemMessage(content="Hi Bob! How can I assist you today?"),
            HumanMessage(content="Can you give me a cool and catchy nickname?")
        ],
        "language": "English"
    },
    config=config_5
)

print("Response in English:", response.content)


Response in English: Sure, Bob! How about "Bobster"?
